<a href="https://colab.research.google.com/github/HalcyonForest/Time-Series-Forecasting/blob/main/HFT_05_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow
import xgboost as xgb
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator # Generates batches for sequence data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,SimpleRNN,LSTM, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import RepeatVector
from keras import optimizers
from keras.layers import Bidirectional
from tensorflow.keras import regularizers
from keras.regularizers import l2
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

In [14]:
def split_sequences(sequences, n_steps_in, n_steps_out):
  X, y = list(), list()
  for i in range(len(sequences)):
  # find the end of this pattern
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out-1
  # check if we are beyond the dataset
    if out_end_ix > len(sequences):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
    X.append(seq_x)
    y.append(seq_y)
  return np.array(X), np.array(y)

def tickSlicing(data, tickRate, windowSize, predictSize):
  window_predict_in_seconds(tickRate, windowSize, predictSize)
  scaler = MinMaxScaler()
  data_scaled = df
  data_scaled[['CLOSE', 'VOL']] = scaler.fit_transform(data_scaled[['CLOSE', 'VOL']])
  data_scaled = data_scaled[::tickRate]
  data = data[::tickRate]
  #data_scaled = data_scaled[::tickRate]
  #print(data_scaled.head())

  X_close = np.reshape(np.array(data_scaled['CLOSE']), (len(data['CLOSE']),-1))
  X_vol = np.reshape(np.array(data_scaled['VOL']), (len(data['VOL']),-1))
  Y_close = np.reshape(np.array(data['CLOSE']), (len(data['CLOSE']), -1))
  data = []
  data = np.hstack((X_close, X_vol, Y_close))
  # Переделать под слайсинг двух параметров - Close + vol upd:DONE
  X, y = split_sequences(data, windowSize, predictSize)
  return X, y, scaler

def y_to_plot(y, y_step):
  y = np.reshape(y[::y_step], (-1))
  return y

def window_predict_in_seconds(tickrate, window, horizon):
  print("Смотрим каждый ", tickrate, "-ый тик, за последние ",tickrate * window / 12 ,"секунд")
  print("предсказываем на ",tickrate * horizon / 12 ," секунды вперед")

def error(X_test, y_test, model):
  y_pred = model.predict(X_test)
  y_pred = np.reshape(y_pred, (y_pred.shape[0], y_pred.shape[1]))
  error = mean_squared_error(y_pred, y_test)
  print(error)
  return error

def error_mae(X_test, y_test, model):
  y_pred = model.predict(X_test)
  y_pred = np.reshape(y_pred, (y_pred.shape[0], y_pred.shape[1]))
  error = mean_absolute_error(y_pred, y_test)
  print(error)
  return error

def plot_history(history_arr):
  for history in history_arr:
    plt.figure(figsize=(6,3))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.legend(['train', 'validation'])
    plt.title('Loss')
    plt.show()

In [15]:
df = pd.read_csv('/content/drive/MyDrive/SBER_HFT_10D.csv')
df['DATE'] = df['<DATE>']
df['TIME'] = df['<TIME>']
df['CLOSE'] = df['<LAST>']
df['VOL'] = df['<VOL>']
df = df.drop(columns=['<DATE>', '<TIME>','<LAST>', '<VOL>'])
df['DATETIME'] = df['DATE'].astype(str) + df['TIME'].astype(str)
df['DATETIME'].head()
time = pd.to_datetime(df['DATETIME'], format='%Y%m%d%H%M%S')
df.index = time
df = df.drop(columns=['DATE', 'TIME', 'DATETIME'])

In [16]:
n_steps_in = 150 
n_steps_out = 14
tickrate = 2
df = df.iloc[:df.shape[0] // 2]
Xe, ye, scaler = tickSlicing(df, tickrate, n_steps_in, n_steps_out)
n_input = Xe.shape[1] * Xe.shape[2]
Xem = np.reshape(Xe, (Xe.shape[0], n_input))
Xm_train,Xm_test,ym_train, ym_test = train_test_split(Xe, ye, test_size=0.4, shuffle=False) 
Xe.shape

Смотрим каждый  2 -ый тик, за последние  25.0 секунд
предсказываем на  2.3333333333333335  секунды вперед


(211110, 150, 2)

In [17]:
def GetModel(nodes, layers, epochs, dropout, X_train, y_train, X_test, y_test):
  model_lstm = Sequential()
  model_lstm.add(LSTM(nodes, activation='relu', input_shape=(n_steps_in, Xe.shape[2]), return_sequences=(layers>1),
                      kernel_regularizer=l2(0.00001), activity_regularizer=l2(0.00001)))
  for i in range(1,layers):
    model_lstm.add(LSTM(nodes // 2, activation='relu', return_sequences=(i < layers-1), recurrent_dropout=0.2, kernel_regularizer=l2(0.00001), 
               activity_regularizer=l2(0.00001) ))
    nodes = nodes // 2
  model_lstm.add(Dense((nodes**(2*layers))*10, activation='relu', kernel_regularizer=l2(0.000001)))
  model_lstm.add(Dropout(dropout))
  model_lstm.add(Dense(n_steps_out, activation='relu', kernel_regularizer=l2(0.000001)))
  optAdam = optimizers.Adam(clipnorm=0.5, learning_rate = 0.0001, clipvalue=0.5)
  # optSGD = optimizers.SGD(lr=0.001, momentum=0.9, clipnorm=0.5, clipvalue=1.0)
  model_lstm.compile(optimizer=optAdam, loss='mae',
    #metrics=[tf.keras.metrics.MeanAbsolutePercentageError]
    )
  history = model_lstm.fit(X_train, y_train, epochs = epochs, verbose=1, batch_size = 1500, validation_data=(X_test, y_test), callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5))
  err_mlp = error_mae(X_test, y_test, model_lstm)
  return history, err_mlp

def NodeSearch(params, X_train, y_train, X_test, y_test):
  hys = []
  errors = []
  for param in params:
    hystory, error = GetModel(param, layers=1, epochs=100, dropout=0.35, X_train=X_train, y_train=y_train , X_test=X_test, y_test=y_test)
    hys.append(hystory)
    errors.append(error)
    print("Nodes: ", param, "MAE error: ", error)
  return hys, errors

def LayersSearch(params, X_train, y_train, X_test, y_test):
  hys = []
  errors = []
  for param in params:
    hystory, error = GetModel(nodes=30, layers=param, epochs=60, dropout=0.35, X_train=X_train, y_train=y_train , X_test=X_test, y_test=y_test)
    hys.append(hystory)
    errors.append(errors)
    print("Layers: ", param, "MAE error: ", error)
  return hys, errors

In [18]:
nodes = [10, 20, 30, 40, 50]
layers = [2,3,4]
epochs = [50,100,150]
dropout = [0.1,0.25,0.4]
batch_size = [10,20,50,100]

In [ ]:
%%time
nodes_hys, nodes_errors = NodeSearch(nodes, Xm_train, ym_train, Xm_test, ym_test)

In [ ]:
%%time
layerss_hys, layers_errors = LayersSearch(layers, Xm_train, ym_train, Xm_test, ym_test)

Epoch 1/60


In [ ]:
print(nodes_errors)
#plot_history(nodes_hys)

In [ ]:
print(layers_errors)
#plot_history(layerss_hys)